<a href="https://colab.research.google.com/github/tomonari-masada/course2022-sml/blob/main/nearest_neighbors_appendix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k近傍法のAppendix

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

* lifesat_extended.csvをアップロードする。

In [2]:
from google.colab import files
files.upload()

Saving lifesat_extended.csv to lifesat_extended (1).csv


{'lifesat_extended.csv': b'Country,Air pollution,Assault rate,Consultation on rule-making,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,Job security,Life expectancy,Life satisfaction,Long-term unemployment rate,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education,GDP per capita\nRussia,15.0,3.8,2.5,15.1,94.0,0.16,69.0,12.8,19292.0,3412.0,11.0,4.0,70.2,6.0,1.7,20885.0,90.0,0.9,37.0,481.0,14.97,65.0,56.0,16.0,9054.914\nTurkey,35.0,5.0,5.5,12.7,34.0,40.86,50.0,1.2,14095.0,3251.0,21.0,8.1,74.6,5.6,2.37,16919.0,86.0,1.1,68.0,462.0,13.42,88.0,62.0,16.4,9437.372\nHungary,15.0,3.6,7.9,4.8,82.0,3.19,58.0,1.3,15442.0,13277.0,20.0,5.7,75.2,4.9,5.1,20948.0,87.0,1.1,57.0,487.0,15.04,62.0,77.0,17.6,1223

In [3]:
df = pd.read_csv('lifesat_extended.csv', index_col=0)

* 日本をtest setとして除外し、残りのデータ集合を使う。
 * 日本の生活満足度を予測することを、最終的な目標としたのだった。

In [4]:
df_test = df.loc[['Japan']]
df = df.drop(['Japan'])

In [5]:
X = df[['GDP per capita', 'Employment rate']]
y = df['Life satisfaction']

* 演習問題5-3: 韓国の生活満足度を予測し、最も良い予測値を与えるkを調べよう。

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = X.copy()
X_scaled[X.columns] = scaler.fit_transform(X)

* 国どうしの距離を保存しておくためのデータフレーム`X_dist`を、`X`をコピーして作っておく。

In [7]:
X_dist = X.copy()

* 韓国と、他の国一つ一つとの距離を求めて、`X_dist`の`dist Korea`という列に保存する。

In [8]:
country = 'Korea'
X_dist['dist ' + country] = np.linalg.norm(X_scaled - X_scaled.loc[country], axis=1)

In [9]:
X_dist

,GDP per capita,Employment rate,dist Korea
Country,,,
Russia,9054.914,69.0,0.416558
Turkey,9437.372,50.0,0.569441
Hungary,12239.894,58.0,0.367957
Poland,12495.334,60.0,0.336988
Slovak Republic,15991.736,60.0,0.268555
Estonia,17288.083,68.0,0.244132
Greece,18064.288,49.0,0.494731
Portugal,19121.592,61.0,0.195163
Slovenia,20732.482,63.0,0.141521


* 韓国に近い順に、国をソートする。

In [10]:
sorted_indices = X_dist.sort_values('dist ' + country).drop(country).index
print(sorted_indices)

Index(['Slovenia', 'Portugal', 'Israel', 'France', 'Spain', 'Estonia', 'Italy',
       'Slovak Republic', 'Belgium', 'Poland', 'New Zealand', 'Finland',
       'Hungary', 'Germany', 'United Kingdom', 'Russia', 'Canada', 'Austria',
       'Netherlands', 'Greece', 'Ireland', 'Australia', 'Turkey', 'Sweden',
       'Denmark', 'United States', 'Iceland'],
      dtype='object', name='Country')


* 近傍の個数`k`を動かしながら、予測誤差を求める。

In [11]:
errors = []
for k in range(1, len(X_scaled)):
  error = abs(y[country] - y[sorted_indices[:k]].mean())
  errors.append(error)
  print(f'\tk={k} error={error:.3f}')

	k=1 error=0.100
	k=2 error=0.400
	k=3 error=0.267
	k=4 error=0.375
	k=5 error=0.440
	k=6 error=0.333
	k=7 error=0.314
	k=8 error=0.313
	k=9 error=0.400
	k=10 error=0.360
	k=11 error=0.464
	k=12 error=0.558
	k=13 error=0.446
	k=14 error=0.500
	k=15 error=0.533
	k=16 error=0.513
	k=17 error=0.571
	k=18 error=0.600
	k=19 error=0.647
	k=20 error=0.565
	k=21 error=0.595
	k=22 error=0.636
	k=23 error=0.600
	k=24 error=0.633
	k=25 error=0.676
	k=26 error=0.704
	k=27 error=0.741


* 韓国の生活満足度予測について、最も小さい予測誤差を与えるkを求める。

In [12]:
k = np.array(errors).argmin() + 1
print(f'When k={k}, we obtain the smallest prediction error for {country}.')

When k=1, we obtain the smallest prediction error for Korea.


* ドイツについて同じことをしてみる。

In [13]:
country = 'Germany'
X_dist['dist ' + country] = np.linalg.norm(X_scaled - X_scaled.loc[country], axis=1)
sorted_indices = X_dist.sort_values('dist ' + country).drop(country).index
errors = []
for k in range(1, len(X_scaled)):
  error = abs(y[country] - y[sorted_indices[:k]].mean())
  errors.append(error)
k = np.array(errors).argmin() + 1
print(f'When k={k}, we obtain the smallest prediction error for {country}.')

When k=17, we obtain the smallest prediction error for Germany.
